In [ ]:
 !pip install contractions
import contractions
import nltk
nltk.download('punkt')
import string
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
import numpy as np
import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import os
#mount google colab
drive.mount('/content/drive',force_remount = True) 
#read the data

Mounted at /content/drive


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Kawther_chatbot/Book 8.xlsx',names = ['question','answer'])

In [ ]:
pd.set_option('display.max_colwidth', None)
data.head(10)

,question,answer
0,Fees Training Goals?,To understand Fees Model.
1,Fees Training Goals?,To have entry points in code to investigate a fee problem or to make evolutions to fees algorithm.
2,Fees Training Goals?,To learn how fees calculation process is working.
3,Fees Training Goals?,To be able to interpret fee results.
4,Fees Training Goals?,To understand fees module interaction with Solife other modules.
5,Fees in Life Insurance world?,"Life insurance can give you and your family financial peace of mind if something were to happen to you. If you’re thinking about buying life insurance, one important factor to consider is its cost."
6,Type of Fees?,Entry Fees
7,Type of Fees?,Management Fees
8,Type of Fees?,Switch Fees
9,Type of Fees?,Exit Fees


In [ ]:
data['question_tokenized'] = data['question'].apply(lambda x: [contractions.fix(word) for word in str(x).split()])
data['answer_tokenized'] = data['answer'].apply(lambda x: [contractions.fix(word) for word in str(x).split()])
data.head()

,question,answer,question_tokenized,answer_tokenized
0,Fees Training Goals?,To understand Fees Model.,"[Fees, Training, Goals?]","[To, understand, Fees, Model.]"
1,Fees Training Goals?,To have entry points in code to investigate a fee problem or to make evolutions to fees algorithm.,"[Fees, Training, Goals?]","[To, have, entry, points, in, code, to, investigate, a, fee, problem, or, to, make, evolutions, to, fees, algorithm.]"
2,Fees Training Goals?,To learn how fees calculation process is working.,"[Fees, Training, Goals?]","[To, learn, how, fees, calculation, process, is, working.]"
3,Fees Training Goals?,To be able to interpret fee results.,"[Fees, Training, Goals?]","[To, be, able, to, interpret, fee, results.]"
4,Fees Training Goals?,To understand fees module interaction with Solife other modules.,"[Fees, Training, Goals?]","[To, understand, fees, module, interaction, with, Solife, other, modules.]"


In [ ]:
data['question_tokenized'] = [' '.join(map(str, l)) for l in data['question_tokenized']]
data['answer_tokenized'] = [' '.join(map(str, l)) for l in data['answer_tokenized']]
data.head()

,question,answer,question_tokenized,answer_tokenized
0,Fees Training Goals?,To understand Fees Model.,Fees Training Goals?,To understand Fees Model.
1,Fees Training Goals?,To have entry points in code to investigate a fee problem or to make evolutions to fees algorithm.,Fees Training Goals?,To have entry points in code to investigate a fee problem or to make evolutions to fees algorithm.
2,Fees Training Goals?,To learn how fees calculation process is working.,Fees Training Goals?,To learn how fees calculation process is working.
3,Fees Training Goals?,To be able to interpret fee results.,Fees Training Goals?,To be able to interpret fee results.
4,Fees Training Goals?,To understand fees module interaction with Solife other modules.,Fees Training Goals?,To understand fees module interaction with Solife other modules.


In [ ]:
data['question_tokenized'] = data['question_tokenized'].apply(nltk.word_tokenize)
data['answer_tokenized'] = data['answer_tokenized'].apply(nltk.word_tokenize)
data.head()

,question,answer,question_tokenized,answer_tokenized
0,Fees Training Goals?,To understand Fees Model.,"[Fees, Training, Goals, ?]","[To, understand, Fees, Model, .]"
1,Fees Training Goals?,To have entry points in code to investigate a fee problem or to make evolutions to fees algorithm.,"[Fees, Training, Goals, ?]","[To, have, entry, points, in, code, to, investigate, a, fee, problem, or, to, make, evolutions, to, fees, algorithm, .]"
2,Fees Training Goals?,To learn how fees calculation process is working.,"[Fees, Training, Goals, ?]","[To, learn, how, fees, calculation, process, is, working, .]"
3,Fees Training Goals?,To be able to interpret fee results.,"[Fees, Training, Goals, ?]","[To, be, able, to, interpret, fee, results, .]"
4,Fees Training Goals?,To understand fees module interaction with Solife other modules.,"[Fees, Training, Goals, ?]","[To, understand, fees, module, interaction, with, Solife, other, modules, .]"


In [ ]:
data['question_tokenized'] = data['question_tokenized'].apply(lambda x: [word.lower() for word in x])
data['answer_tokenized'] = data['answer_tokenized'].apply(lambda x: [word.lower() for word in x])
data.head()

,question,answer,question_tokenized,answer_tokenized
0,Fees Training Goals?,To understand Fees Model.,"[fees, training, goals, ?]","[to, understand, fees, model, .]"
1,Fees Training Goals?,To have entry points in code to investigate a fee problem or to make evolutions to fees algorithm.,"[fees, training, goals, ?]","[to, have, entry, points, in, code, to, investigate, a, fee, problem, or, to, make, evolutions, to, fees, algorithm, .]"
2,Fees Training Goals?,To learn how fees calculation process is working.,"[fees, training, goals, ?]","[to, learn, how, fees, calculation, process, is, working, .]"
3,Fees Training Goals?,To be able to interpret fee results.,"[fees, training, goals, ?]","[to, be, able, to, interpret, fee, results, .]"
4,Fees Training Goals?,To understand fees module interaction with Solife other modules.,"[fees, training, goals, ?]","[to, understand, fees, module, interaction, with, solife, other, modules, .]"


In [ ]:
punc = string.punctuation
data['question_tokenized'] = data['question_tokenized'].apply(lambda x: [word for word in x if word not in punc])
data.head()

,question,answer,question_tokenized,answer_tokenized
0,Fees Training Goals?,To understand Fees Model.,"[fees, training, goals]","[to, understand, fees, model, .]"
1,Fees Training Goals?,To have entry points in code to investigate a fee problem or to make evolutions to fees algorithm.,"[fees, training, goals]","[to, have, entry, points, in, code, to, investigate, a, fee, problem, or, to, make, evolutions, to, fees, algorithm, .]"
2,Fees Training Goals?,To learn how fees calculation process is working.,"[fees, training, goals]","[to, learn, how, fees, calculation, process, is, working, .]"
3,Fees Training Goals?,To be able to interpret fee results.,"[fees, training, goals]","[to, be, able, to, interpret, fee, results, .]"
4,Fees Training Goals?,To understand fees module interaction with Solife other modules.,"[fees, training, goals]","[to, understand, fees, module, interaction, with, solife, other, modules, .]"


In [ ]:
def clean(x):
  for i,word in enumerate(x):
    wordNew = re.sub('[\-_,.`"\']', '', word)
    if len(wordNew)<1:
      x.remove(word)
    elif word != wordNew:
      x[i] = wordNew
  return x

data['question_tokenized'].apply(lambda x: clean(x))
data['answer_tokenized'].apply(lambda x: clean(x))
data.head()

,question,answer,question_tokenized,answer_tokenized
0,Fees Training Goals?,To understand Fees Model.,"[fees, training, goals]","[to, understand, fees, model]"
1,Fees Training Goals?,To have entry points in code to investigate a fee problem or to make evolutions to fees algorithm.,"[fees, training, goals]","[to, have, entry, points, in, code, to, investigate, a, fee, problem, or, to, make, evolutions, to, fees, algorithm]"
2,Fees Training Goals?,To learn how fees calculation process is working.,"[fees, training, goals]","[to, learn, how, fees, calculation, process, is, working]"
3,Fees Training Goals?,To be able to interpret fee results.,"[fees, training, goals]","[to, be, able, to, interpret, fee, results]"
4,Fees Training Goals?,To understand fees module interaction with Solife other modules.,"[fees, training, goals]","[to, understand, fees, module, interaction, with, solife, other, modules]"


In [ ]:
data.head(50)

,question,answer,question_tokenized,answer_tokenized
0,Fees Training Goals?,To understand Fees Model.,"[fees, training, goals]","[to, understand, fees, model]"
1,Fees Training Goals?,To have entry points in code to investigate a fee problem or to make evolutions to fees algorithm.,"[fees, training, goals]","[to, have, entry, points, in, code, to, investigate, a, fee, problem, or, to, make, evolutions, to, fees, algorithm]"
2,Fees Training Goals?,To learn how fees calculation process is working.,"[fees, training, goals]","[to, learn, how, fees, calculation, process, is, working]"
3,Fees Training Goals?,To be able to interpret fee results.,"[fees, training, goals]","[to, be, able, to, interpret, fee, results]"
4,Fees Training Goals?,To understand fees module interaction with Solife other modules.,"[fees, training, goals]","[to, understand, fees, module, interaction, with, solife, other, modules]"
5,Fees in Life Insurance world?,"Life insurance can give you and your family financial peace of mind if something were to happen to you. If you’re thinking about buying life insurance, one important factor to consider is its cost.","[fees, in, life, insurance, world]","[life, insurance, can, give, you, and, your, family, financial, peace, of, mind, if, something, were, to, happen, to, you, if, you, are, thinking, about, buying, life, insurance, one, important, factor, to, consider, is, its, cost]"
6,Type of Fees?,Entry Fees,"[type, of, fees]","[entry, fees]"
7,Type of Fees?,Management Fees,"[type, of, fees]","[management, fees]"
8,Type of Fees?,Switch Fees,"[type, of, fees]","[switch, fees]"
9,Type of Fees?,Exit Fees,"[type, of, fees]","[exit, fees]"


In [ ]:
word_count = {}
for question in data['question_tokenized']:
    for word in question:
        if word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1
for answer in data['answer_tokenized']:
    for word in answer:
        if word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1
print(len(word_count))
print(word_count)

260
{'fees': 82, 'training': 5, 'goals': 5, 'in': 13, 'life': 3, 'insurance': 3, 'world': 1, 'type': 13, 'of': 7, 'solife': 4, 'computation': 4, 'results': 8, 'initialization': 1, 'nan': 2, 'endorsedfee': 5, 'parameters': 10, 'object': 5, 'status': 3, 'definition': 5, 'evaluation': 8, 'the': 23, 'result': 7, 'compute': 1, 'fini': 1, 'nst': 1, 'generic': 3, 'step': 2, 'accounting': 4, 'action': 10, 'charging': 2, 'update': 1, 'payment': 7, 'scheduled': 18, 'impact': 5, 'analysis': 5, 'and': 21, 'tests': 5, 'how': 12, 'to': 35, 'test': 11, 'modification': 11, 'understand': 2, 'model': 1, 'have': 1, 'entry': 2, 'points': 1, 'code': 1, 'investigate': 1, 'a': 5, 'fee': 15, 'problem': 1, 'or': 3, 'make': 1, 'evolutions': 1, 'algorithm': 2, 'learn': 1, 'calculation': 3, 'process': 2, 'is': 22, 'working': 1, 'be': 3, 'able': 1, 'interpret': 1, 'module': 1, 'interaction': 1, 'with': 5, 'other': 2, 'modules': 1, 'can': 1, 'give': 1, 'you': 3, 'your': 1, 'family': 1, 'financial': 12, 'peace': 1, 

In [ ]:
threshold= 2
words_indexed = {}
word_number = 4

tokens = ['<PAD>', '<OUT>', '<SOS>', '<EOS>']
for token in tokens:
    words_indexed[token] = len(words_indexed)

for word, count in word_count.items():
    if count >= threshold:
        words_indexed[word] = word_number
        word_number += 1

sizeOfVocab = len(words_indexed)
print(sizeOfVocab)
print(words_indexed)


124
{'<PAD>': 0, '<OUT>': 1, '<SOS>': 2, '<EOS>': 3, 'fees': 4, 'training': 5, 'goals': 6, 'in': 7, 'life': 8, 'insurance': 9, 'type': 10, 'of': 11, 'solife': 12, 'computation': 13, 'results': 14, 'nan': 15, 'endorsedfee': 16, 'parameters': 17, 'object': 18, 'status': 19, 'definition': 20, 'evaluation': 21, 'the': 22, 'result': 23, 'generic': 24, 'step': 25, 'accounting': 26, 'action': 27, 'charging': 28, 'payment': 29, 'scheduled': 30, 'impact': 31, 'analysis': 32, 'and': 33, 'tests': 34, 'how': 35, 'to': 36, 'test': 37, 'modification': 38, 'understand': 39, 'entry': 40, 'a': 41, 'fee': 42, 'or': 43, 'algorithm': 44, 'calculation': 45, 'process': 46, 'is': 47, 'be': 48, 'with': 49, 'other': 50, 'you': 51, 'financial': 52, 'if': 53, 'are': 54, 'its': 55, 'switch': 56, 'as': 57, ':': 58, 'proposition': 59, 'endorsedpolicy': 60, 'at': 61, 'several': 62, 'amounts': 63, 'for': 64, 'actions': 65, 'operation': 66, 'market': 67, 'when': 68, 'event': 69, '(': 70, ')': 71, 'default': 72, 'each'

In [ ]:
for question in data['question_tokenized']:
    question.append('<EOS>')
    question.insert(0,'<SOS>')

for answer in data['answer_tokenized']:
    answer.append('<EOS>')
    answer.insert(0,'<SOS>')

In [ ]:
print(data['question_tokenized'])

0                       [<SOS>, fees, training, goals, <EOS>]
1                       [<SOS>, fees, training, goals, <EOS>]
2                       [<SOS>, fees, training, goals, <EOS>]
3                       [<SOS>, fees, training, goals, <EOS>]
4                       [<SOS>, fees, training, goals, <EOS>]
5            [<SOS>, fees, in, life, insurance, world, <EOS>]
6                              [<SOS>, type, of, fees, <EOS>]
7                              [<SOS>, type, of, fees, <EOS>]
8                              [<SOS>, type, of, fees, <EOS>]
9                              [<SOS>, type, of, fees, <EOS>]
10                           [<SOS>, fees, in, solife, <EOS>]
11                 [<SOS>, fees, computation, results, <EOS>]
12                 [<SOS>, fees, computation, results, <EOS>]
13                 [<SOS>, fees, computation, results, <EOS>]
14                       [<SOS>, fees, initialization, <EOS>]
15                                        [<SOS>, nan, <EOS>]
16      

In [ ]:
def convertToInt(inputSen):
    global words_indexed
    sen_into_int = []
    for word in inputSen:
        if word not in words_indexed:
            sen_into_int.append(words_indexed['<OUT>'])
        else:
            sen_into_int.append(words_indexed[word])
    return sen_into_int

data['question_tokenized_int'] = data['question_tokenized'].apply(lambda x: convertToInt(x))
data['answer_tokenized_int'] = data['answer_tokenized'].apply(lambda x: convertToInt(x))

In [ ]:
data.head(50)

,question,answer,question_tokenized,answer_tokenized,question_tokenized_int,answer_tokenized_int
0,Fees Training Goals?,To understand Fees Model.,"[<SOS>, fees, training, goals, <EOS>]","[<SOS>, to, understand, fees, model, <EOS>]","[2, 4, 5, 6, 3]","[2, 36, 39, 4, 1, 3]"
1,Fees Training Goals?,To have entry points in code to investigate a fee problem or to make evolutions to fees algorithm.,"[<SOS>, fees, training, goals, <EOS>]","[<SOS>, to, have, entry, points, in, code, to, investigate, a, fee, problem, or, to, make, evolutions, to, fees, algorithm, <EOS>]","[2, 4, 5, 6, 3]","[2, 36, 1, 40, 1, 7, 1, 36, 1, 41, 42, 1, 43, 36, 1, 1, 36, 4, 44, 3]"
2,Fees Training Goals?,To learn how fees calculation process is working.,"[<SOS>, fees, training, goals, <EOS>]","[<SOS>, to, learn, how, fees, calculation, process, is, working, <EOS>]","[2, 4, 5, 6, 3]","[2, 36, 1, 35, 4, 45, 46, 47, 1, 3]"
3,Fees Training Goals?,To be able to interpret fee results.,"[<SOS>, fees, training, goals, <EOS>]","[<SOS>, to, be, able, to, interpret, fee, results, <EOS>]","[2, 4, 5, 6, 3]","[2, 36, 48, 1, 36, 1, 42, 14, 3]"
4,Fees Training Goals?,To understand fees module interaction with Solife other modules.,"[<SOS>, fees, training, goals, <EOS>]","[<SOS>, to, understand, fees, module, interaction, with, solife, other, modules, <EOS>]","[2, 4, 5, 6, 3]","[2, 36, 39, 4, 1, 1, 49, 12, 50, 1, 3]"
5,Fees in Life Insurance world?,"Life insurance can give you and your family financial peace of mind if something were to happen to you. If you’re thinking about buying life insurance, one important factor to consider is its cost.","[<SOS>, fees, in, life, insurance, world, <EOS>]","[<SOS>, life, insurance, can, give, you, and, your, family, financial, peace, of, mind, if, something, were, to, happen, to, you, if, you, are, thinking, about, buying, life, insurance, one, important, factor, to, consider, is, its, cost, <EOS>]","[2, 4, 7, 8, 9, 1, 3]","[2, 8, 9, 1, 1, 51, 33, 1, 1, 52, 1, 11, 1, 53, 1, 1, 36, 1, 36, 51, 53, 51, 54, 1, 1, 1, 8, 9, 1, 1, 1, 36, 1, 47, 55, 1, 3]"
6,Type of Fees?,Entry Fees,"[<SOS>, type, of, fees, <EOS>]","[<SOS>, entry, fees, <EOS>]","[2, 10, 11, 4, 3]","[2, 40, 4, 3]"
7,Type of Fees?,Management Fees,"[<SOS>, type, of, fees, <EOS>]","[<SOS>, management, fees, <EOS>]","[2, 10, 11, 4, 3]","[2, 1, 4, 3]"
8,Type of Fees?,Switch Fees,"[<SOS>, type, of, fees, <EOS>]","[<SOS>, switch, fees, <EOS>]","[2, 10, 11, 4, 3]","[2, 56, 4, 3]"
9,Type of Fees?,Exit Fees,"[<SOS>, type, of, fees, <EOS>]","[<SOS>, exit, fees, <EOS>]","[2, 10, 11, 4, 3]","[2, 1, 4, 3]"


In [ ]:
def converToTensor(question,answer):
    question = question.to_tensor()
    answer = answer.to_tensor()
    return question,answer

In [ ]:
#data[data.columns[0:2]].values
#testingData=tf.ragged.constant(data[data.columns[4:6]].values)
#data

In [ ]:
#print(testingData[0][1])

In [ ]:
#testingOutput=[([2,5,3],[2,4,1]),([2,8,3],[2,4,3])]
#hh=tf.convert_to_tensor(testingOutput)
#print(type(hh[0]))
#testingTensor = tf.ragged.constant(testingOutput)
#print(type(testingTensor))
#print(testingTensor[0])
#datasetNew = tf.data.Dataset.from_tensor_slices(testingTensor)
#for x in testingData:
#  print(x)
#  break
#print(testingData.to_tensor())

In [ ]:
#datasetNew = tf.data.Dataset.from_tensor_slices(testingData)
#tensorData = testingData.to_tensor()
#tensorData
#datasetNew = tf.data.Dataset.from_tensor_slices(tensorData)
#tensorData=tf.expand_dims(tensorData,axis=1)
#tensorData
#for x in tensorData:
  #print(tf.reshape(x, [,]))
  #print(tf.expand_dims(x,axis=0))
 # print(x)
 # break
#datasetNew = tf.reshape(testingData, [,])
#print(testingData.get_shape())

In [ ]:
#print(type(data['question_tokenized_int'].values))
listOfQuestionsNpArray=data['question_tokenized_int'].values
listOfAnswersNpArray=data['answer_tokenized_int'].values
tensorQuestions=tf.ragged.constant(listOfQuestionsNpArray,dtype=tf.int64)
tensorAnswers=tf.ragged.constant(listOfAnswersNpArray,dtype=tf.int64)
datasetNew = tf.data.Dataset.from_tensor_slices((tensorQuestions,tensorAnswers))


In [ ]:
def convertSentenceToTensor(sentence):
  sentence = [contractions.fix(word) for word in str(sentence).split()]
  #print(sentence)
  sentence = ' '.join(map(str, sentence))
  #print(sentence)
  sentence = str(sentence).split()
  #print(sentence)
  sentence = [word.lower() for word in sentence]
  #print(sentence)
  sentence = [word for word in sentence if word not in punc]
  #print(sentence)
  sentence = clean(sentence)
  #print(sentence)
  sentence.append('<EOS>')
  sentence.insert(0,'<SOS>')
  #print(sentence)
  sentence = convertToInt(sentence)
  #print(sentence)
  sentence = tf.convert_to_tensor(sentence, dtype=tf.int64)
  #print(sentence)
  sentence = tf.expand_dims(sentence, 0)
  print(sentence)
  return sentence

In [ ]:
convertSentenceToTensor("Hi, I'm robot and my name is \"John\" ! ? $")

tf.Tensor([[ 2  1  1  1  1 33  1  1 47  1  3]], shape=(1, 11), dtype=int64)


<tf.Tensor: shape=(1, 11), dtype=int64, numpy=array([[ 2,  1,  1,  1,  1, 33,  1,  1, 47,  1,  3]])>

In [ ]:
#for x in datasetNew:
#  print(x)
#  break

In [ ]:

#datasetNew = tf.data.Dataset.from_tensor_slices(hh)
#print(type(datasetNew))
#for x in datasetNew:
#  print(x)
#  break

In [ ]:
BUFFER_SIZE = 2500
BATCH_SIZE = 6 #you can change it 

In [ ]:
def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(converToTensor, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE))


train_batches = make_batches(datasetNew)

In [ ]:
for x in train_batches:
  print("outB:")
  print(x)
  break

outB:
(<tf.Tensor: shape=(6, 7), dtype=int64, numpy=
array([[ 2, 30,  4,  3,  0,  0,  0],
       [ 2,  4,  5,  6,  3,  0,  0],
       [ 2, 35, 36, 37,  4, 38,  3],
       [ 2, 22,  4, 21, 23,  3,  0],
       [ 2,  4,  5,  6,  3,  0,  0],
       [ 2, 35, 36, 37,  4, 38,  3]])>, <tf.Tensor: shape=(6, 18), dtype=int64, numpy=
array([[  2,  30,   4,  54, 102,  57,  95,   1,  66, 111,   1,  48, 112,
          1,  55,   1,   1,   3],
       [  2,  36,  39,   4,   1,   3,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0],
       [  2,  42,  63,  54,  93,  33, 121,   7,  52,  66,  26, 117,   3,
          0,   0,   0,   0,   0],
       [  2,  22,   4,  21,  23,  91,  47,  92,  36,   1,  42,  63,  93,
         61,  52,  66,  80,   3],
       [  2,  36,   1,  35,   4,  45,  46,  47,   1,   3,   0,   0,   0,
          0,   0,   0,   0,   0],
       [  2,  72,   4,  54, 118, 119,   3,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0]])>)


In [ ]:
#print(type(train_batches))
#for x in train_batches:
#  print("outPT:")
#  print(x[0][0])
#  print("outEN:")
#  print(x[1][0])
#  print("out:")
#  print(fromIntToWords(tf.expand_dims(x[0][0],axis=0)))
#  print("out2:")
#  print(fromIntToWords(tf.expand_dims(x[1][0],axis=0)))
#  break

Transformer architecture source: https://www.tensorflow.org/text/tutorials/transformer

In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)


In [ ]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights
def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print('Attention weights are:')
  print(temp_attn)
  print('Output is:')
  print(temp_out)
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, answer_input, question_target, rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, answer_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                           target_vocab_size, question_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inp, tar, training, enc_padding_mask,
           look_ahead_mask, dec_padding_mask):

    enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)

    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

    return final_output, attention_weights

In [ ]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [ ]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=2))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

In [ ]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=sizeOfVocab + 1,
    target_vocab_size=sizeOfVocab + 1,
    answer_input=1000,
    question_target=1000,
    rate=dropout_rate)

In [ ]:
def create_masks(inp, tar):
  # Encoder padding mask
  enc_padding_mask = create_padding_mask(inp)

  # Used in the 2nd attention block in the decoder.
  # This padding mask is used to mask the encoder outputs.
  dec_padding_mask = create_padding_mask(inp)

  # Used in the 1st attention block in the decoder.
  # It is used to pad and mask future tokens in the input received by
  # the decoder.
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

  return enc_padding_mask, combined_mask, dec_padding_mask

In [ ]:
#checkpoint_path = "./checkpoints2/train"

checkpoint_path = "/content/drive/MyDrive/kawther_chatbot/checkpoints1_real/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [ ]:
EPOCHS = 10

In [ ]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),#tf.int64
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]


@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]

  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp,
                                 True,
                                 enc_padding_mask,
                                 combined_mask,
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)
  train_accuracy(accuracy_function(tar_real, predictions))

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  train_loss.reset_states()
  train_accuracy.reset_states()

  for (batch, (inp, tar)) in enumerate(train_batches):
    train_step(inp, tar)

    if batch % 50 == 0:
      print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

Epoch 1 Batch 0 Loss 0.2867 Accuracy 0.8986
Epoch 1 Loss 0.2558 Accuracy 0.9064
Time taken for 1 epoch: 19.68 secs

Epoch 2 Batch 0 Loss 0.1866 Accuracy 0.9265
Epoch 2 Loss 0.2394 Accuracy 0.8996
Time taken for 1 epoch: 0.55 secs

Epoch 3 Batch 0 Loss 0.1752 Accuracy 0.9205
Epoch 3 Loss 0.2177 Accuracy 0.9128
Time taken for 1 epoch: 0.55 secs

Epoch 4 Batch 0 Loss 0.2365 Accuracy 0.8814
Epoch 4 Loss 0.2136 Accuracy 0.9064
Time taken for 1 epoch: 0.54 secs

Epoch 5 Batch 0 Loss 0.1477 Accuracy 0.9512
Saving checkpoint for epoch 5 at /content/drive/MyDrive/checkpoints1_real/train/ckpt-43
Epoch 5 Loss 0.1954 Accuracy 0.9200
Time taken for 1 epoch: 1.00 secs

Epoch 6 Batch 0 Loss 0.2078 Accuracy 0.9032
Epoch 6 Loss 0.2183 Accuracy 0.9197
Time taken for 1 epoch: 0.56 secs

Epoch 7 Batch 0 Loss 0.2159 Accuracy 0.8750
Epoch 7 Loss 0.2833 Accuracy 0.9101
Time taken for 1 epoch: 0.55 secs

Epoch 8 Batch 0 Loss 0.3978 Accuracy 0.8906
Epoch 8 Loss 0.2701 Accuracy 0.9001
Time taken for 1 epoch: 0.

In [ ]:
#ckpt_save_path = ckpt_manager.save()

In [ ]:
def returnTokens():
  start = tf.convert_to_tensor(2, dtype=tf.int64)
  end = tf.convert_to_tensor(3, dtype=tf.int64)
  return start, end

In [ ]:
print(returnTokens()[0])

tf.Tensor(2, shape=(), dtype=int64)


In [ ]:
inv_words_indexed = {value:key for key, value in words_indexed.items()}
def fromIntToWords(output):
  sentence = []
  for number in output[0]:
    #print(inv_words_indexed[number])
    #with tf.Session() as sess:  print(number.eval())
    word = inv_words_indexed[number.numpy()]
    if number.numpy() not in [1,2,3]:
      sentence.append(word)
  sentence = " ".join(sentence)
  return sentence

In [ ]:
fromIntToWords(convertSentenceToTensor("Hi, this is test."))

tf.Tensor([[ 2  1 87 47 37  3]], shape=(1, 6), dtype=int64)


'this is test'

In [ ]:
def evaluate(sentence, max_length=40):

  sentence = convertSentenceToTensor(sentence)
  encoder_input = sentence
  start, end = returnTokens()
  output = tf.convert_to_tensor([start])
  output = tf.expand_dims(output, 0)

  for i in range(max_length):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)

    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(encoder_input,
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.argmax(predictions, axis=-1)
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)
    # return the result if the predicted_id is equal to the end token
    if predicted_id == end:
      break

  # output.shape (1, tokens)
  text = fromIntToWords(output)  # shape: ()
  return text

In [ ]:
def print_translation(sentence, tokens):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens}')

In [ ]:
sentence = "what are fees"

translated_text = evaluate(sentence)
print_translation(sentence, translated_text)

tf.Tensor([[ 2  1 54  4  3]], shape=(1, 5), dtype=int64)
Input:         : what are fees
Prediction     : when the fee event ( switch ) default fee for each type are evaluated and endorsedfees are created endorsedfees are created in status


In [ ]:
sentence = "Training Goals?"

translated_text = evaluate(sentence)
print_translation(sentence, translated_text)

tf.Tensor([[2 5 1 3]], shape=(1, 4), dtype=int64)
Input:         : Training Goals?
Prediction     : when the fee event ( switch ) default fee for each type are evaluated and endorsedfees are created endorsedfees are created in status


In [ ]:
sentence = "when fees are applied"

translated_text = evaluate(sentence)
print_translation(sentence, translated_text)

tf.Tensor([[  2  68   4  54 110   3]], shape=(1, 6), dtype=int64)
Input:         : when fees are applied
Prediction     : fees are in as : proposition or endorsedpolicy
